In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel,PeftConfig
import re
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch
from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type,
)

model_type1 = ModelType.mistral_7b_chat_v2
template_type1 = get_default_template_type(model_type1)
model_mistral, tokenizer_mistral = get_model_tokenizer(model_type1, model_kwargs={'device_map': 'auto'}, model_dir="/home/css/models/Mistral-7B-Instruct-v0.2")

import json
def validate_format(input_string):
    pattern = re.compile(r'^\d{4}-\d{2}$')
    return bool(pattern.match(input_string))

def sql_json_to_alpaca_json(json_path, new_data_list):
    # 读取已有的 JSON 数据
    try:
        with open(json_path, "r") as file:
            json_data_list = json.load(file)
    except FileNotFoundError:
        # 如果文件不存在，创建一个空的列表
        json_data_list = []

    # 将新数据追加到列表
    json_data_list.extend(new_data_list)

    # 将更新后的列表写回 JSON 文件
    with open(json_path, "w") as json_file:
        json.dump(json_data_list, json_file, indent=4)


[INFO:swift] Setting torch_dtype: torch.bfloat16


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.16s/it]


In [7]:

import re




#推理结果
tune_file_path = '/home/qiuyang/workplace/swift/examples/pytorch/llm/my_tuning/data/mistral_list2.json'
# 打开文件并加载JSON数据
with open(tune_file_path, 'r') as file:
    tune_result_list = json.load(file)

print(len(tune_result_list))

# 打开JSON文件
i = 0
new_data = [] 
with open('/home/qiuyang/workplace/swift/examples/pytorch/llm/my_tuning/data/c4_data2.json', 'r') as file:
    # 逐行读取文件内容
    for line in file:
        # 解析JSON数据
        data = json.loads(line)
        content = data['content']
        if len(tokenizer_mistral.tokenize(data['content'])) <3500:
            tune_result = tune_result_list[i]
            i = i + 1
            tune_result = str(tune_result)
            result = tune_result[1:-1].split(',')

            if len(result) == 3:
                output = "[No,UNKNOWN,UNKNOWN,UNKNOWN]"
                new_json_data = {
                    'instruction': '''Whether anyone in the information was transferred between countries in text?if so,from which country to which country?What year and month did the transfer take place?only give the starting point and destination example [Yes or No,starting point,destination,year-month],just country name.''',
                    'input': content,
                    'output': output
                }
                new_data.append(new_json_data)
            elif len(result) == 4:
                if result[0] == "Yes" and validate_format(result[3]):
                    output = tune_result
                    new_json_data = {
                    'instruction': '''Whether anyone in the information was transferred between countries in text?if so,from which country to which country?What year and month did the transfer take place?only give the starting point and destination example [Yes or No,starting point,destination,year-month],just country name.''',
                    'input': content,
                    'output': output
                }
                new_data.append(new_json_data)
        
print(i)
sql_json_to_alpaca_json('/home/qiuyang/llama-recipes/src/llama_recipes/lla_datasets/alpaca_data_siyuanzu_process.json',new_data_list=new_data)


10215
10215
